In [ ]:
import os

import numpy as np
import pandas as pd

evaluation_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'signals-2024-04-20')
predictions_file = os.path.join(evaluation_dir, 'predictions.csv')

predictions = pd.read_csv(predictions_file)
predictions['signal'] = predictions['signal'].apply(eval).apply(np.array)
predictions['execution_time'] = pd.to_timedelta(predictions['execution_time'])

ground_truth_file = os.path.join(evaluation_dir, 'ground_truth.csv')
ground_truth = pd.read_csv(ground_truth_file)
ground_truth['signal'] = ground_truth['signal'].apply(eval).apply(np.array)

predictions.head()

## Frequency extraction evaluation

In [ ]:
import respiration.analysis as analysis

lowpass = 0.08
highpass = 0.6

bpm_results = []

for index, row in predictions.iterrows():
    subject, scenario = row['subject'], row['scenario']

    # Get the ground truth signal and sampling rate
    gt_signal, gt_sampling_rate = ground_truth[
        (ground_truth['subject'] == subject) &
        (ground_truth['scenario'] == scenario)
        ][['signal', 'sampling_rate']].values[0]

    gt_extractor = analysis.FrequencyExtractor(
        gt_signal,
        gt_sampling_rate,
        lowpass,
        highpass)

    gt_frequencies = {
        'fft': gt_extractor.frequency_from_fft(),
        'pc': gt_extractor.frequency_from_peaks(),
        'cp': gt_extractor.frequency_from_crossing_point(),
        'nfcp': gt_extractor.frequency_from_nfcp(),
    }

    frequency_extractor = analysis.FrequencyExtractor(
        row['signal'],
        row['sampling_rate'],
        lowpass,
        highpass)

    predicted_frequencies = {
        'fft': frequency_extractor.frequency_from_fft(),
        'pc': frequency_extractor.frequency_from_peaks(),
        'cp': frequency_extractor.frequency_from_crossing_point(),
        'nfcp': frequency_extractor.frequency_from_nfcp(),
    }

    for method in gt_frequencies.keys():
        # The frequency is in Hz, we want to convert it to bpm
        predicted_bpm = predicted_frequencies[method] * 60
        gt_bpm = gt_frequencies[method] * 60

        bpm_results.append({
            'subject': subject,
            'scenario': scenario,
            'strategy': row['method'],
            'roi': row['roi'],
            'method': method,
            'gt_bpm': predicted_bpm,
            'predicted_bpm': gt_bpm,
            'error_bpm': round(abs(gt_bpm - predicted_bpm), 2),
        })

In [ ]:
bpm_results_df = pd.DataFrame(bpm_results)
bpm_results_df

## Distance evaluation

In [ ]:
import respiration.preprocessing as preprocessing

distance_results = []

for index, row in predictions.iterrows():
    subject, scenario = row['subject'], row['scenario']

    # Get the ground truth signal and sampling rate
    gt_signal, gt_sampling_rate = ground_truth[
        (ground_truth['subject'] == subject) &
        (ground_truth['scenario'] == scenario)
        ][['signal', 'sampling_rate']].values[0]

    gt_processed = preprocessing.standard_processing(gt_signal, gt_sampling_rate)
    gt_signal = preprocessing.resample_signal(gt_processed, len(row['signal']))

    prediction_processed = preprocessing.standard_processing(row['signal'], row['sampling_rate'])

    distance_results.append({
        'subject': subject,
        'scenario': scenario,
        'strategy': row['method'],
        'roi': row['roi'],
        'distance_mse': analysis.distance_mse(gt_signal, prediction_processed),
        'distance_euclidean': analysis.distance_euclidean(gt_signal, prediction_processed),
        'pearson_correlation': analysis.pearson_correlation(gt_signal, prediction_processed),
    })

In [ ]:
distance_df = pd.DataFrame(distance_results)
distance_df['distance_mse'] = abs(distance_df['distance_mse'])
distance_df['distance_euclidean'] = abs(distance_df['distance_euclidean'])
distance_df['pearson_correlation'] = abs(distance_df['pearson_correlation'])

distance_df

In [ ]:
# Average the pearson correlation for each strategy
of_pearson = distance_df[
    (distance_df['strategy'] == 'optical_flow') &
    (distance_df['roi'] == 'chest')]['pearson_correlation'].mean()

pi_pearson = distance_df[
    (distance_df['strategy'] == 'pixel_intensity') &
    (distance_df['roi'] == 'chest')]['pearson_correlation'].mean()

mtts_pearson = distance_df[distance_df['strategy'] == 'mtts_can']['pearson_correlation'].mean()

print(f'optical_flow correlation:    {of_pearson:.2f}')
print(f'pixel_intensity correlation: {pi_pearson:.2f}')
print(f'mtts_can correlation:        {mtts_pearson:.2f}')

In [ ]:
distance_df[distance_df['strategy'] == 'mtts_can']